Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.30.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacitypgml
udacityplaygroundrg
japaneast
653662f1-95de-4498-b876-1fd625bf5d18


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'udacityproject2'
project_folder = './pipeline-bike-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
udacityproject2,udacitypgml,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "udacityproject2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bikeno"
description_text = "Bike Sharing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,instant,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,2.997264,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,2.004787,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,1.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,3.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,5.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,6.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

,instant,date,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01,1,2011-01-01,1,0,1,6,2,0.344167,0.363625,0.805833,0.160446,331,654,985
2011-01-02,2,2011-01-02,1,0,1,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2011-01-03,3,2011-01-03,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
2011-01-04,4,2011-01-04,1,0,1,2,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
2011-01-05,5,2011-01-05,1,0,1,3,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'normalized_root_mean_squared_error',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "forecasting",
                             training_data=dataset,
                             time_column_name="date", 
                             label_column_name="cnt",  
                             path = project_folder,
                             enable_early_stopping= True,
                             #featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [88046a15][f3698302-a65c-4e5e-90c9-eceb98a76d5a], (This step will run and generate new outputs)
Submitted PipelineRun e4db0505-897c-49d8-87be-e150c6b73550
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e4db0505-897c-49d8-87be-e150c6b73550?wsid=/subscriptions/653662f1-95de-4498-b876-1fd625bf5d18/resourcegroups/udacityplaygroundrg/workspaces/udacitypgml&tid=60e63508-9cfd-4013-9a1f-7610707b07ce


In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
pipeline_run.wait_for_completion()

PipelineRunId: e4db0505-897c-49d8-87be-e150c6b73550
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e4db0505-897c-49d8-87be-e150c6b73550?wsid=/subscriptions/653662f1-95de-4498-b876-1fd625bf5d18/resourcegroups/udacityplaygroundrg/workspaces/udacitypgml&tid=60e63508-9cfd-4013-9a1f-7610707b07ce
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 7c13552e-6cf3-4ea1-a143-009ba8bb11dc
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/7c13552e-6cf3-4ea1-a143-009ba8bb11dc?wsid=/subscriptions/653662f1-95de-4498-b876-1fd625bf5d18/resourcegroups/udacityplaygroundrg/workspaces/udacitypgml&tid=60e63508-9cfd-4013-9a1f-7610707b07ce
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '7c13552e-6cf3-4ea1-a143-009ba8bb11dc', 'target': 'udacityproject2', 'status': 'Completed', 'startTimeUtc': '2021-07-14T06:12:58.562554Z', 'endT

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [14]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/7c13552e-6cf3-4ea1-a143-009ba8bb11dc/metrics_data, 1 files out of an estimated total of 1


In [15]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_1,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_20,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_23,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_27,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_19,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_22,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_25,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_4,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_12,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_14,...,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_8,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_6,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_5,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_10,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_13,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_15,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_17,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_28,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_30,7c13552e-6cf3-4ea1-a143-009ba8bb11dc_32
r2_score,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],...,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
normalized_root_mean_squared_log_error,[1.8575194295079226e-05],[0.017050914932548174],[0.011206611086424755],[0.01761007978473286],[0.01737775082236407],[0.01737775082236407],[1.2040924163875155e-07],[0.12709691599346445],[0.005815167940773748],[0.00383692101324394],...,[3.3166254560389575e-07],[0.060407419929974185],[0.05413379748284958],[0.015027794416149116],[0.03112370164017534],[0.01366466277992622],[0.014146533929136474],[0.011089234646635755],[8.850305639934937e-05],[0.01581805359682461]
normalized_mean_absolute_error,[2.5737099526532125e-05],[0.026783673105838096],[0.015326225903768465],[0.02413343629576948],[0.024188663249259876],[0.024188663249259876],[1.5409804953297425e-07],[0.26447434221752675],[0.007148386182617775],[0.00452921147049019],...,[4.247070633954834e-07],[0.08499332147579457],[0.07603379133521793],[0.021215429913336713],[0.039982457456256115],[0.02233435541281518],[0.01963066417909952],[0.01685004466581847],[0.00011343738235796933],[0.02072845342518605]
explained_variance,[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],...,[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0],[1.0]
root_mean_squared_log_error,[0.00011028662632259767],[0.10123651218671252],[0.06653708755868148],[0.10455644543349205],[0.10317703711885215],[0.10317703711885215],[7.149066021838734e-07],[0.7546133762185778],[0.034526435820763444],[0.02278097700055053],...,[1.969182268091174e-06],[0.3586573816185693],[0.3214089607663372],[0.08922462510476734],[0.18479096358498664],[0.08113129445075362],[0.08399231127374698],[0.06584018763136737],[0.0005254697934496732],[0.09391663626613908]
normalized_root_mean_squared_error,[2.5737099526532125e-05],[0.026783673105838096],[0.015326225903768465],[0.02413343629576948],[0.024188663249259876],[0.024188663249259876],[1.5409804953297425e-07],[0.26447434221752675],[0.007148386182617775],[0.00452921147049019],...,[4.247070633954834e-07],[0.08499332147579457],[0.07603379133521793],[0.021215429913336713],[0.039982457456256115],[0.02233435541281518],[0.01963066417909952],[0.01685004466581847],[0.00011343738235796933],[0.02072845342518605]
normalized_median_absolute_error,[2.5737099526532125e-05],[0.026783673105838096],[0.015326225903768465],[0.02413343629576948],[0.024188663249259876],[0.024188663249259876],[1.5409804953297425e-07],[0.26447434221752675],[0.007148386182617775],[0.00452921147049019],...,[4.247070633954834e-07],[0.08499332147579457],[0.07603379133521793],[0.021215429913336713],[0.039982457456256115],[0.02233435541281518],[0.01963066417909952],[0.01685004466581847],[0.00011343738235796933],[0.02072845342518605]
mean_absolute_percentage_error,[0.011032673091018507],[10.177552128158805],[6.837712999486412],[10.811331363873176],[10.764507193205713],[10.764507193205713],[7.153195308087802e-05],[122.54409211402658],[3.5643990130762786],[2.332435530255835],...,[0.0001970320837406983],[33.405147376672936],[26.212179210428694],[9.153887811458096],[17.677243515547048],[8.024240915032127],[8.729712077960773],[6.578864198030724],[0.05259537801513261],[9.460484057629099]
ro

### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/7c13552e-6cf3-4ea1-a143-009ba8bb11dc/model_data, 1 files out of an estimated total of 1


In [17]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(country_or_region=None, drop_column_names=[], featurization_config=FeaturizationConfig(
    blocked_transformers=None,
    column_purposes=None,
    transformer_params=None,
    dataset_language=None,
    drop_columns=None
), force_time_index_features=None, freq='D', gr...
    timeseries_param_dict={'time_column_name': 'date', 'grain_column_names': None, 'drop_column_names': [], 'overwrite_columns': True, 'dropna': False, 'transform_dictionary': {'min': '_automl_target_col', 'max': '_automl_target_col', 'mean': '_automl_target_col'}, 'max_horizon': 1, 'origin_time_colname': 'origin', 'country_or_region': None, 'n_cross_validations': 5, 'short_series_handling': True, 'max_cores_per_iteration': 1, 'feature_lags': None, 'target_aggregation_function': None, 'seasonality': 7, 'use_st

In [18]:
best_model.steps

[('timeseriestransformer',
  TimeSeriesTransformer(
      pipeline_type=TimeSeriesPipelineType.FULL,
      max_horizon=1,
      use_stl=None,
      seasonality=7,
      force_time_index_features=None,
      grain_column_names=['_automl_dummy_grain_col'],
      drop_column_names=[],
      origin_time_colname='origin',
      group=None,
      country_or_region=None,
      freq='D',
      time_column_name='date',
      pipeline=Pipeline(
          memory=None,
          steps=[('make_numeric_na_dummies', MissingDummiesTransformer(
              numerical_columns=['instant', 'season', 'yr', 'mnth', 'weekday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']
          )), ('impute_na_numeric_datetime', TimeSeriesImputer(
              end=None,
              freq='D',
              impute_by_horizon=False,
              input_column=['instant', 'season', 'yr', 'mnth', 'weekday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered'],
           

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [19]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacitypgml
udacityplaygroundrg
japaneast
653662f1-95de-4498-b876-1fd625bf5d18


In [20]:
# NOTE: update these to match your existing experiment name and a previous experiment
experiment_name = 'udacityproject2'
project_folder = './pipeline-bike-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
udacityproject2,udacitypgml,Link to Azure Machine Learning studio,Link to Documentation


In [27]:
from azureml.pipeline.core import PipelineRun

run_id = "e4db0505-897c-49d8-87be-e150c6b73550"
pipeline_run = PipelineRun(experiment, run_id)

In [28]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bikesharing Train", description="Training bikesharing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bikesharing Train,14cfcca6-46c1-4149-810d-e01010a201f9,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [29]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [30]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bike-rest-endpoint"}
                        )

In [31]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  75a69e35-4cbb-469e-bd1f-9534abbd73cc


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [32]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bike-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …